In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from itertools import product
from scipy.stats import pearsonr



In [2]:
class Dummy_obj:
    def __init__(self, onnx, vnnlib, n_splits, ot, res_o, lambda_t, lambda_res):
        self.onnx, self.vnnlib, self.n_splits = onnx, vnnlib, int(n_splits)
        self.ot, self.res_o = float(ot), res_o
        self.lambda_t, self.lambda_res = lambda_t, lambda_res
        self.skip = True if n_splits==-1 else False
        
        
        
    def __str__(self):
        return f"Instance: {self.onnx}, {self.vnnlib}, {self.n_splits}, {self.res_o}, {self.ot}, {self.lambda_res}, {self.lambda_t}"
    

In [3]:
def load_csv(file):
    with open(file) as f:
        # get the lambda values
        header = f.readline()
        lambdas = header.split(",")[7:]
        lambdas = [i for i in lambdas if "_t" in i]
        lambdas = [float(i.split("_")[0]) for i in lambdas]
#         print(lambdas)
        
        rows = []
        for l in f:
            fileds = l.split(',')
            assert len(fileds)==27
            if fileds[4] in ["SKIPPED", "SPLIT_FAILED"]:
                rows.append( Dummy_obj(fileds[0], fileds[1], -1, -1, -1, [], [] ))
                continue
                
            # build lambda dict
            lambda_t = []
            lambda_res = []
            for i in range(7, len(header.split(",")) , 2):
                lambda_res.append(fileds[i])
                lambda_t.append(float(fileds[i+1]))
#             print(lambda_t)
#             print(lambda_res)
            rows.append( Dummy_obj(fileds[0], fileds[1], fileds[4], fileds[6], fileds[5], lambda_t, lambda_res ))
    return (file.stem, rows, lambdas)
            

In [4]:
exp2_path = Path("/Users/linhan/relusplitter/experiment/results/exp4")

def get_ress(benchmark_str):
    res_abcrown = []
    res_nsat    = []
    res_nnenum  = []

    for f in exp2_path.iterdir():
        if f.suffix != ".csv":
            continue
        if "marabou" in f.stem:
            continue
        if benchmark_str not in f.stem:
            continue

        if "abcrown" in f.stem:
            res_abcrown.append(f)
        elif "neuralsat" in f.stem:
            res_nsat.append(f)
        elif "nnenum" in f.stem:
            res_nnenum.append(f)

    #     results.append(load_csv(f))
    #     print(f.stem)
    func = lambda x: x.stem.split("~")[-2]+x.stem.split("~")[-3]
    res_abcrown.sort(key=func)
    res_nsat.sort(key=func)
    res_nnenum.sort(key=func)
    print(len(res_abcrown))
    print(len(res_nsat))
    print(len(res_nnenum))
    return res_abcrown, res_nsat, res_nnenum
    


In [5]:
def check_lambda(fp, gen_talbe=True):
    fname, dps, lambdas = load_csv(fp)
    lambda_t_total = [0 for _ in lambdas]
    cnt = 0
    for dp in dps:
        if len(set([i for i in [dp.res_o] + dp.lambda_res if i in ["sat", "unsat"]])) > 1:
            # Check for conflicting result
            continue
        if dp.skip:
            continue
        lambda_t_total = [a + b for a, b in zip(lambda_t_total, dp.lambda_t)]
        cnt += 1
        
    print("==============================")
    print(fp)
    print(cnt)
    print(lambdas)
    print(lambda_t_total)
    
    # Calculate averages for lambda_t_total
    avg_lambda_t_total = [i/cnt for i in lambda_t_total]
    print(avg_lambda_t_total)
    
    # Check correlation between lambdas and avg_lambda_t_total
    correlation, p_value = pearsonr(lambdas, avg_lambda_t_total)
    
    print(f"Pearson correlation coefficient: {correlation}")
    print(f"P-value: {p_value}")
    
    if gen_talbe:
        # Calculate averages for lambda_t_total
        print(" & ".join([str(i) for i in lambdas]))
        avg_lambda_t_total = [round(i/cnt, 2) for i in lambda_t_total]
        print(" & ".join([str(i) for i in avg_lambda_t_total]))

    
    return lambda_t_total, cnt, lambdas

# ACAS XU

In [6]:
res_abcrown, res_nsat, res_nnenum = get_ress("acas")

1
1
1


In [7]:
lambda_t_total, cnt, lambdas = check_lambda(res_abcrown[0])
lambda_t_total, cnt, lambdas = check_lambda(res_nsat[0])
lambda_t_total, cnt, lambdas = check_lambda(res_nnenum[0])

/Users/linhan/relusplitter/experiment/results/exp4/acasxu~abcrown~stable~random~1~lambda.csv
185
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[553.1386494636536, 528.5515027046204, 552.3865172863007, 552.6985158920288, 552.8683381080627, 553.6096367835999, 553.9837222099304, 552.8306589126587, 553.3945982456207, 552.1432781219482]
[2.9899386457494788, 2.857035149754705, 2.985873066412436, 2.987559545362318, 2.9884775032868256, 2.992484523154594, 2.9945066065401647, 2.988273831960317, 2.991322152679031, 2.9845582601186393]
Pearson correlation coefficient: 0.20061893499595126
P-value: 0.5783866017385952
0.001 & 0.01 & 0.1 & 0.2 & 0.4 & 0.8 & 1.0 & 2.0 & 4.0 & 8.0
2.99 & 2.86 & 2.99 & 2.99 & 2.99 & 2.99 & 2.99 & 2.99 & 2.99 & 2.98
/Users/linhan/relusplitter/experiment/results/exp4/acasxu~neuralsat~stable~random~1~lambda.csv
184
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[1419.5996000000005, 1400.7104000000002, 1404.1417999999996, 1402.605900000001, 1404.0745000000002, 

# MNIST_FC

In [8]:
res_abcrown, res_nsat, res_nnenum = get_ress("mnist")

1
1
1


In [9]:
lambda_t_total, cnt, lambdas = check_lambda(res_abcrown[0])
lambda_t_total, cnt, lambdas = check_lambda(res_nsat[0])
lambda_t_total, cnt, lambdas = check_lambda(res_nnenum[0])

/Users/linhan/relusplitter/experiment/results/exp4/mnist_fc~abcrown~stable~random~1~lambda.csv
64
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[4947.509738683701, 4823.5628781318665, 4710.380713701248, 4712.106866836548, 4723.447039604187, 4706.656981706619, 4717.385422468185, 4707.559799194336, 4713.69163608551, 4708.949820995331]
[77.30483966693282, 75.36816997081041, 73.599698651582, 73.62666979432106, 73.80385999381542, 73.54151533916593, 73.7091472260654, 73.5556218624115, 73.6514318138361, 73.57734095305204]
Pearson correlation coefficient: -0.33160856117512655
P-value: 0.34924850005078695
0.001 & 0.01 & 0.1 & 0.2 & 0.4 & 0.8 & 1.0 & 2.0 & 4.0 & 8.0
77.3 & 75.37 & 73.6 & 73.63 & 73.8 & 73.54 & 73.71 & 73.56 & 73.65 & 73.58
/Users/linhan/relusplitter/experiment/results/exp4/mnist_fc~neuralsat~stable~random~1~lambda.csv
85
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[10923.745299999999, 10035.189599999998, 10268.1404, 9739.7479, 10057.025400000002, 9970.1702, 100

# TLL

In [10]:
res_abcrown, res_nsat, res_nnenum = get_ress("tll")

1
1
0


In [11]:
lambda_t_total, cnt, lambdas = check_lambda(res_abcrown[0])
lambda_t_total, cnt, lambdas = check_lambda(res_nsat[0])

/Users/linhan/relusplitter/experiment/results/exp4/tllverifybench~abcrown~stable~random~1~lambda.csv
32
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[336.8283648490906, 334.6800682544708, 334.88661456108093, 334.9158864021301, 335.3400752544403, 333.93287086486816, 331.247305393219, 330.4325656890869, 330.99208521842957, 331.3249979019165]
[10.52588640153408, 10.458752132952213, 10.46520670503378, 10.466121450066566, 10.47937735170126, 10.43540221452713, 10.351478293538094, 10.326017677783966, 10.343502663075924, 10.35390618443489]
Pearson correlation coefficient: -0.6512228624471613
P-value: 0.041385423059841345
0.001 & 0.01 & 0.1 & 0.2 & 0.4 & 0.8 & 1.0 & 2.0 & 4.0 & 8.0
10.53 & 10.46 & 10.47 & 10.47 & 10.48 & 10.44 & 10.35 & 10.33 & 10.34 & 10.35
/Users/linhan/relusplitter/experiment/results/exp4/tllverifybench~neuralsat~stable~random~1~lambda.csv
32
[0.001, 0.01, 0.1, 0.2, 0.4, 0.8, 1.0, 2.0, 4.0, 8.0]
[305.2282, 302.3483999999999, 301.06589999999994, 302.0854, 301.604, 30

In [12]:
print(len(lambda_t_total))
print(len(lambdas))

10
10
